In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import geopandas as gpd
import os
import urllib
from shapely.geometry import LinearRing, Polygon, Point
import pickle
from itertools import product

%matplotlib inline

In [2]:
# load images for all bands plus the color image
# note: matplotlib can only handle png natively
# pip install pillow to add support for more image types

dir_name = "monterey_data"
gcs_spectral_bucket = "https://storage.googleapis.com/w210data/data/spectral/"
gcs_bathymetry_bucket = "https://storage.googleapis.com/w210data/data/bathymetry/"
gcs_biomass_bucket = "https://storage.googleapis.com/w210data/data/kelp_biomass/"

if not os.path.isdir(dir_name):
    os.mkdir(dir_name)
    
def download_file(file_name, gcs_bucket):
    full_path = os.path.join(dir_name, file_name)
    if not os.path.exists(full_path):
        urllib.request.urlretrieve (gcs_bucket+file_name, full_path)

def load_image(file_name, gcs_bucket):
    download_file(file_name, gcs_bucket)
    full_path = os.path.join(dir_name, file_name)
    return plt.imread(full_path)

b4_file = "LC08_L1TP_044034_20180601_20180614_01_T1_B4.TIF" # red
b5_file = "LC08_L1TP_044034_20180601_20180614_01_T1_B5.TIF" # infrared
color_file = "LC08_L1TP_044034_20180601_20180614_01_T1.jpg"

b4 = load_image(b4_file, gcs_spectral_bucket)
b5 = load_image(b5_file, gcs_spectral_bucket)
color = load_image(color_file, gcs_spectral_bucket)

In [3]:
#Take one vertice and create a square with increment length
def squarify(point, increment):
    return Polygon([(point[0], point[1]),
     (point[0] + increment, point[1]), 
     (point[0] + increment, point[1] + increment), 
     (point[0], point[1] + increment)])

#input corner verticies - top left and bot right, returns bottom corner verticies
def desquarify(left_vertices, right_vertices, square_length):
    x_num = round((right_vertices[0] - left_vertices[0])/square_length, 0)
    y_num = round((left_vertices[1] - right_vertices[1])/square_length, 0)
    x_lengths = np.linspace(left_vertices[0], right_vertices[0]-square_length, x_num)
    y_lengths = np.linspace(left_vertices[1], right_vertices[1]-square_length, y_num)
    return [p for p in product(x_lengths, y_lengths)]


In [4]:
def sat_to_gdf(top_left, bottom_right, b5, b4):
    """
    Function that converts a satellite image to a dataframe with square polygons and polygon centroid points as the summary geometries
    top_left: satellite big top left vertice
    bottom_right: satellite bottom right left vertice
    b4, b5: satellite images bands 4 and 5
    square_length: desired resolution for square polygons in which to cut the satellite images by
    """
    #Convert band 4 and 5 to floats, calculate NDVI
    ir_float = b5.astype(float)
    red_float = b4.astype(float)
    ndvi = np.divide(ir_float - red_float, ir_float + red_float)
    
    #Create the granular grid where each array point has a point value for NDVI
    sq_x = (top_left[1]-bottom_right[1])/ir_float.shape[0]
    sq_y = (bottom_right[0] - top_left[0])/ir_float.shape[1]
    x_lengths = np.linspace(bottom_right[1],top_left[1]-sq_x, ir_float.shape[0])
    y_lengths = np.linspace(bottom_right[0],top_left[0]-sq_x, ir_float.shape[1])
    
    #Every ndvi value should have a point value
    ndvi_column = ndvi.reshape(-1)
    sat_geometry = [Point(p) for p in product(x_lengths, y_lengths)]
    gdf = gpd.GeoDataFrame(ndvi_column, geometry = sat_geometry, crs = {'init' : 'epsg:4326'})
    gdf.columns = ["NDVI", "geometry"]
    gdf.NDVI = np.nan_to_num(gdf.NDVI)
    return gdf

In [5]:
top_left = (-123.39111, 38.52671)
bottom_right = (-120.77835, 36.40801)
square_length = 0.01

mont_sat = sat_to_gdf(top_left, bottom_right, b5, b4)
mont_sat.head()

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:12: RuntimeWarning: divide by zero encountered in true_divide
  if sys.path[0] == '':
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:12: RuntimeWarning: invalid value encountered in true_divide
  if sys.path[0] == '':


,NDVI,geometry
0,0.0,POINT (36.40801 -120.77835)
1,0.0,POINT (36.40801 -120.7786911268399)
2,0.0,POINT (36.40801 -120.7790322536797)
3,0.0,POINT (36.40801 -120.7793733805196)
4,0.0,POINT (36.40801 -120.7797145073595)


In [12]:
def poly_squarify(top_left, bottom_right, square_length):
    """Builds a grid of square polygons each of square_length filling a rectangular area bounded by top_left and bottom_right verticies"""
    return gpd.GeoDataFrame(geometry = [squarify(i, square_length) for i in desquarify(top_left, bottom_right, square_length)], crs = {'init':'epsg:4326'})

In [11]:
square_df = poly_squarify(top_left, bottom_right, square_length)
square_df.head()

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:12: DeprecationWarning: object of type <class 'float'> cannot be safely interpreted as an integer.
  if sys.path[0] == '':
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:13: DeprecationWarning: object of type <class 'float'> cannot be safely interpreted as an integer.
  del sys.path[0]


,geometry
0,"POLYGON ((-123.39111 38.52671, -123.39011 38.5..."
1,"POLYGON ((-123.39111 38.525709197356, -123.390..."
2,"POLYGON ((-123.39111 38.52470839471199, -123.3..."
3,"POLYGON ((-123.39111 38.52370759206799, -123.3..."
4,"POLYGON ((-123.39111 38.52270678942399, -123.3..."


In [ ]:
#sjoin, this may take a long long time
master = gpd.sjoin(square_df, mont_sat, op = "contains", how = "left")
master.head()

In [ ]:
#It's going to be super inflated, but it'll be dissolved to unique level polygons
master.shape

In [ ]:
#then groupby/dissolve
master_grouped = master.dissolve('centroid', as_index=False, aggfunc = np.mean)
master_grouped.head()

In [ ]:
master_grouped["centroid"] = master_grouped.geometry.apply(lambda x: x.centroid.wkt)
master_grouped.drop(["geometry"], inplace = True)
master_grouped.head()

In [ ]:
master_grouped.to_pickle("./monterey_satellite_gdf.pkl")